In [67]:
import pandas as pd
import scipy as sc
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [81]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")
train = train.sample(frac=0.1, random_state=42)

In [48]:
print float(len(train['item_id'].values))/len(sc.unique(train['item_id'].values))
print len(train[train['year'] == 2012])

25.9713261649
21


In [57]:
def SMAPE(target, prediction):
    return 200*np.mean(np.abs(target-prediction)/(np.abs(target) + np.abs(prediction)))

In [101]:
def validate(estimator, train_data):
    def do_stuff(indexes):
        estimator.fit(X[X['year'] < 2014], y[X['year'] < 2014])
    pred1 = estimator.predict(X[X['year'] == 2014])
    score1 = SMAPE(pred1, y[X['year'] == 2014])
    print "trained on 2012, 2013, prediction for 2014: " + str(score1)
    X = train_data.drop(['Num','y'], axis=1)
    y = train_data['y']
    estimator.fit(X[X['year'] < 2014], y[X['year'] < 2014])
    pred1 = estimator.predict(X[X['year'] == 2014])
    score1 = SMAPE(pred1, y[X['year'] == 2014])
    print "trained on 2012, 2013, prediction for 2014: " + str(score1)
    pred2 = estimator.predict(X[X['year'] == 2015])
    score2 = 
    print "trained on 2012, 2013, prediction for 2015: " + str(SMAPE(pred2, y[X['year'] == 2015]))
    estimator.fit(X[X['year'] < 2015], y[X['year'] < 2015])
    pred3 = estimator.predict(X[X['year'] == 2015])
    print "trained on 2012 - 2014 prediction for 2015: " + str(SMAPE(pred3, y[X['year'] == 2015]))
    estimator.fit(X[X['year'] == 2014], y[X['year'] == 2014])
    pred4 = estimator.predict(X[X['year'] == 2015])
    print "trained on 2014 prediction for 2015: " + str(SMAPE(pred4, y[X['year'] == 2015]))

In [96]:
ohe = OneHotEncoder(sparse=False)
dummy_features = ohe.fit_transform(train['item_id'].values.reshape(-1, 1))
train.index = dummy_features.index
dummy_features = pd.DataFrame(dummy_features, columns=['good = ' + str(x) for x in range(dummy_features.shape[1])])
new_train = pd.concat([train, dummy_features], axis=1)

In [117]:
def test_alg(loss, n_estimators=50):
    print 'loss: ' + loss 
    print 'estimators: ' + str(n_estomators)
    from sklearn.ensemble import GradientBoostingRegressor
    model = GradientBoostingRegressor(loss=loss, n_estimators=n_estimators, max_depth=5, random_state=43)
    print 'raw: '
    validate(model, train)
    print 'modified: '
    validate(model, new_train.drop(['item_id'], axis=1))

In [116]:
for i in [1, 5, 10, 15, 20, 30, 40, 50]:
test_alg('lad')


lad
raw: 
trained on 2012, 2013, prediction for 2014: 32.0979079454
trained on 2012, 2013, prediction for 2015: 67.1508803249
trained on 2012 - 2014 prediction for 2015: 78.1886687202
trained on 2014 prediction for 2015: 83.8649757733
modified: 
trained on 2012, 2013, prediction for 2014: 31.1559459311
trained on 2012, 2013, prediction for 2015: 69.6617026431
trained on 2012 - 2014 prediction for 2015: 75.7325021779
trained on 2014 prediction for 2015: 81.2163945089
